In [6]:
from keras.optimizers import SGD,Adam
from keras.layers import Input
from keras.models import Model
import numpy as np
import math
from PIL import Image
from matplotlib.pyplot import imshow
from layers import ReflectionPadding2D,Denormalize,conv_bn_relu,res_conv,dconv_bn_relu

%matplotlib inline

%reload_ext autoreload

%autoreload 2


In [7]:
def image_transform_net():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res_conv(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    a = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    # Scale output to range [0, 255] via custom Denormalize layer
    y = Denormalize(name='transform_output')(a)
    return  Model(input=x, output=y)

print image_transform_net().summary() 


# Input 3 × 256 × 256
# Reflection Padding (40 × 40) 3 × 336 × 336
# 32 × 9 × 9 conv, stride 1 32 × 336 × 336
# 64 × 3 × 3 conv, stride 2 64 × 168 × 168
# 128 × 3 × 3 conv, stride 2 128 × 84 × 84
# Residual block, 128 filters 128 × 80 × 80
# Residual block, 128 filters 128 × 76 × 76
# Residual block, 128 filters 128 × 72 × 72
# Residual block, 128 filters 128 × 68 × 68
# Residual block, 128 filters 128 × 64 × 64
# 64 × 3 × 3 conv, stride 1/2 64 × 128 × 128
# 32 × 3 × 3 conv, stride 1/2 32 × 256 × 256
# 3 × 9 × 9 conv, stride 1 3 × 256 × 256

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
reflectionpadding2d_4 (Reflectio (None, 336, 336, 3)   0           input_4[0][0]                    
____________________________________________________________________________________________________
convolution2d_53 (Convolution2D) (None, 336, 336, 32)  7808        reflectionpadding2d_4[0][0]      
____________________________________________________________________________________________________
batchnormalization_27 (BatchNorm (None, 336, 336, 32)  128         convolution2d_53[0][0]           
___________________________________________________________________________________________

In [8]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg16
from keras import backend as K


Image Util

In [18]:
def loss_net(x_in, trux_x_in):
    # Append the initial input to the FastNet input to the VGG inputs
    x = merge([x_in, trux_x_in], mode='concat', concat_axis=0)

    # Normalize the inputs via custom VGG Normalization layer
    x = VGGNormalize(name="vgg_normalize")(x)
    
    
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res_conv(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    y = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    return  Model(input=x, output=y)














model = VGG(self.img_height, self.img_width).append_vgg_model(model.input, x_in=model.output,
                                                                          pool_type=self.pool_type)

            if self.model is None:
                self.model = model

            self.vgg_output_dict = dict([(layer.name, layer.output) for layer in model.layers[-18:]])

            vgg_layers = dict([(layer.name, layer) for layer in model.layers[-18:]])

            style = img_utils.preprocess_image(style_image_path, self.img_width, self.img_height)
            print('Getting style features from VGG network.')

            self.style_layers = ['conv1_2', 'conv2_2', 'conv3_3', 'conv4_3']

            self.style_layer_outputs = []
            for layer in self.style_layers:
                self.style_layer_outputs.append(self.vgg_output_dict[layer])

            style_features = self.get_vgg_style_features(style)
            self.style_features = style_features

            # Style Reconstruction Loss
            if self.style_weight != 0.0:
                for i, layer_name in enumerate(self.style_layers):
                    layer = vgg_layers[layer_name]
                    style_loss = StyleReconstructionRegularizer(
                        style_feature_target=style_features[i][0],
                        weight=self.style_weight)(layer)

                    layer.add_loss(style_loss)

            # Feature Reconstruction Loss
            self.content_layer = 'conv4_2'
            self.content_layer_output = self.vgg_output_dict[self.content_layer]

            if self.content_weight != 0.0:
                layer = vgg_layers[self.content_layer]
                content_regularizer = FeatureReconstructionRegularizer(
                    weight=self.content_weight)(layer)
                layer.add_loss(content_regularizer)

        # Total Variation Regularization
        if self.tv_weight != 0.0:
            layer = fastnet_output_layer  # Fastnet Output layer
            tv_regularizer = TVRegularizer(img_width=self.img_width, img_height=self.img_height,
                                           weight=self.tv_weight)(layer)
            layer.add_loss(tv_regularizer)

        if self.model is None:
            self.model = model
        return model


In [122]:
base_image_path = 
# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))
    

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# build the VGG16 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)


# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)


def loss_net():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    y = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    return  Model(input=x, output=y)

NameError: name 'base_image_path' is not defined